# Life Expectancy Visualization using Folium

The notebook aims to demonstrate how Folium maps can be used for data visualization and exporting the resulting map objects to GIF format, with a specific focus on life expectancy across countries.

### Import Modules

In [ ]:
import pandas as pd
import numpy as np
import os
import folium
from html2image import Html2Image
import imageio.v2 as imageio
import warnings
warnings.filterwarnings('ignore')

## Prepare Data
To prepare the data for visualization, the first step is to load the dataset, and then fill any null or missing values with zero.

In [ ]:
df_life = pd.read_csv('data/gapminder/life_expectancy.csv', index_col=0)
df_life_2022 = df_life[['2022']].reset_index()
df_life_2022.fillna(0, inplace=True)

## Generating and Saving Folium Map

Folium is a Python library for creating interactive maps. It works by rendering map data as HTML, CSS, and JavaScript and can display various types of map data, including markers, lines, and polygons. To create a choropleth map using Folium, we need to provide a GeoJSON file that describes the boundaries of the regions we want to display. In this case, I slightly modified the version of the [countries.geojson](https://raw.githubusercontent.com/datasets/geo-boundaries-world-110m/master/countries.geojson) file from the Geo Boundaries dataset, which contains the geographic boundaries of countries around the world. 

Please note some countries may have imperfect shapes, and gray areas indicate the absence of data (e.g., Kosovo).

In [ ]:
m = folium.Map(location=[0,0], zoom_start=2, zoom_control=False)

folium.Choropleth(
    geo_data='data/gapminder/world-countries.json',
    name='choropleth',
    data=df_life_2022,
    columns=['country', '2022'],
    key_on='properties.name',
    fill_opacity=0.8,
    line_opacity=0.5,
    fill_color='YlGnBu',
    highlight=False,
    smooth_factor=0.9,
    legend_name='Life Expectancy in 1920',
    threshold_scale=np.linspace(0, 100, 20).tolist(),
).add_to(m)

m

The code snippet below is to visualizing life expectancy on a world map using Folium and saving the map as HTML and PNG files

In [ ]:
def visualize_map(input_year):
    custom_df = df_life[[input_year]].reset_index()
    custom_df.fillna(0, inplace=True)
    m = folium.Map(location=[0,0], zoom_start=2, zoom_control=False)
    folium.Choropleth(
        geo_data='data/gapminder/world-countries.json',
        name='choropleth',
        data=custom_df,
        columns=['country', input_year],
        key_on='properties.name',
        fill_opacity=0.8,
        line_opacity=0.5,
        fill_color='YlGnBu',
        highlight=False,
        smooth_factor=0.5,
        legend_name=f'Life Expectancy in {input_year}',
        threshold_scale=np.linspace(0, 100, 20).tolist()
    ).add_to(m)

    # You can't save the PNG directly from the map object, so we save the map as an HTML file
    html_file = f"./output/life_expectancy/{input_year}.html"
    m.save(html_file)

    # Save map as PNG image using html2image
    hti = Html2Image()
    hti.output_path = './output/life_expectancy/'
    hti.screenshot(html_file=html_file, save_as=f"{input_year}.png")
    return m

Run this code snippet below to test the function. The process may take up to 5 minutes to complete, covering the years 1800 to 2050.

In [ ]:
for year in range(1800, 2051, 1):
    year=str(year)
    visualize_map(year)

This code snippets saves the exported PNG to GIF.

In [ ]:
output_dir = 'output/life_expectancy'
images_from_dir = [os.path.join(output_dir, item) for item in os.listdir(output_dir) if item.endswith('.png')]
images_from_dir.sort()

crop_px = 200
images = []
for file_path in images_from_dir:
    img = imageio.imread(file_path)
    cropped_img = img[:-crop_px, :, :]
    images.append(cropped_img)

gif_path = './output/life_expectancy.gif'
imageio.mimsave(gif_path, images, fps=12, loop=0)


## Conclusion

![Alt Text](./output/life_expectancy.gif)

The life expectancy data provides valuable insights into the progress of human health and well-being over time. Although there has been a consistent and stable growth since 1800, with some fluctuations during major historical events (Spanish Influenza and World War II), the rate of growth varies across subregions. While some regions have made significant progress, others are lagging behind, particularly in Sub-Saharan Africa and Asia. It is crucial to recognize these disparities and take action to address them. This can include investing in healthcare infrastructure, education, and public health initiatives. Ultimately, the goal should be to achieve global health equity, where everyone has an equal opportunity for a long and healthy life.

## Learning Resources

1. Academis by Dr. Kristian Rother [https://github.com/krother/](https://github.com/krother/Python3_Basics_Tutorial/blob/master/data_analysis/gapminder.md)
2. Gapminder [https://www.gapminder.org/](https://www.gapminder.org/tag/life-expectancy/)
3. Folium Documentation. [https://python-visualization.github.io/folium](https://python-visualization.github.io/folium/quickstart.html)
4. DataHub [https://github.com/datasets](https://github.com/datasets)